# Load dataset

In [2]:
import pandas as pd
df = pd.read_csv('../data/athletes.csv')

KeyboardInterrupt: 

In [20]:
%store -r df

### Create Response Variable

In [22]:
# There are Na values in some columns. Fill with 0
df['total_lift'] = df['candj'].fillna(0) + df['snatch'].fillna(0) + df['deadlift'].fillna(0) + df['backsq'].fillna(0)

### Split and Model

In [23]:
from sklearn.model_selection import train_test_split

vars = ['region','gender', 'eat', 'background', 'experience', 'schedule', 'howlong','age','height','weight','candj','snatch','deadlift','backsq']
cats = ['region','gender', 'eat', 'background', 'experience', 'schedule', 'howlong']
numcs = ['age','height','weight']

# x = pd.get_dummies(x, columns=cats) # encode categorical variables

x = df[numcs].fillna(0) # NAs in numeric columns, fill 0 if any

y = df['total_lift']

# train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [4]:
import tensorflow as tf
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer

# model hyperparameters
epochs = 10
batch_size = 32

# Define privacy parameters
l2_norm_clip = 1.0  # Clipping norm
noise_multiplier = 0.5  # Noise multiplier
num_microbatches = 1  # Number of microbatches
learning_rate = 0.1  # Learning rate

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(x_train.shape[1],)),   # Input layer for the number of features
    tf.keras.layers.Dense(64, activation='relu'),       # Hidden layer 1
    tf.keras.layers.Dense(32, activation='relu'),       # Hidden layer 2
    tf.keras.layers.Dense(1)                            # Output layer (for regression)
])

# Create the optimizer with differential privacy
optimizer = DPGradientDescentGaussianOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)  # Use `dp_sum_query` for better privacy guarantees.


In [5]:
# Compile the model
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

NameError: name 'model' is not defined

In [24]:
import tensorflow as tf
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy import DPKerasAdamOptimizer

# DP parameters
l2_norm_clip = 1
noise_multiplier = 0.5

# model hyperparameters
epochs = 10
batch_size = 32

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(x_train.shape[1],)),   # Input layer for the number of features
    tf.keras.layers.Dense(64, activation='relu'),       # Hidden layer 1
    tf.keras.layers.Dense(32, activation='relu'),       # Hidden layer 2
    tf.keras.layers.Dense(1)                            # Output layer (for regression)
])

optimizer = DPKerasAdamOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier
    )

loss = tf.keras.losses.MeanSquaredError(
    reduction=tf.losses.Reduction.NONE,
    name='mean_squared_error'
)


In [25]:
model.compile(optimizer=optimizer, loss=loss)

model.fit(x_train, y_train,
          epochs=5,
          batch_size=32)

Epoch 1/5
751/751 [==============================] - 8s 6ms/step - loss: 144811.0312
Epoch 2/5
751/751 [==============================] - 5s 7ms/step - loss: 48011.8945
Epoch 3/5
751/751 [==============================] - 5s 6ms/step - loss: 46869.7422
Epoch 4/5
751/751 [==============================] - 5s 7ms/step - loss: 46250.2852
Epoch 5/5
751/751 [==============================] - 5s 6ms/step - loss: 45950.0938


### Compute Metrics

In [26]:
# Evaluate the model
test_loss = model.evaluate(x_test, y_test)
print(f'Test loss: {test_loss}')

# Make predictions
y_pred = model.predict(x_test)

188/188 [==============================] - 0s 2ms/step - loss: 45189.7461
Test loss: 45189.74609375
188/188 [==============================] - 0s 2ms/step


In [27]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Compute regression metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)  # Calculate RMSE

# Display the metrics summary
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Mean Absolute Error (MAE): 164.74
Mean Squared Error (MSE): 45212.21
Root Mean Squared Error (RMSE): 212.63


### Compute DP epsilon

In [28]:
print(compute_dp_sgd_privacy.compute_dp_sgd_privacy_statement(x_train.shape[0],
                                    batch_size=256,
                                    noise_multiplier=noise_multiplier,
                                    num_epochs=epochs,
                                    delta=.00001
                                    ))

DP-SGD performed over 24023 examples with 256 examples per iteration, noise
multiplier 0.5 for 10 epochs with microbatching, and no bound on number of
examples per user.

This privacy guarantee protects the release of all model checkpoints in addition
to the final model.

Example-level DP with add-or-remove-one adjacency at delta = 1e-05 computed with
RDP accounting:
    Epsilon with each example occurring once per epoch:       138.688
    Epsilon assuming Poisson sampling (*):                    139.347

No user-level privacy guarantee is possible without a bound on the number of
examples per user.

(*) Poisson sampling is not usually done in training pipelines, but assuming
that the data was randomly shuffled, it is believed the actual epsilon should be
closer to this value than the conservative assumption of an arbitrary data
order.

